In [1]:
import pandas as pd
import torch
from torch import nn
from ipdb import set_trace
import torchvision as tv
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader as DL
from utils.OID_Dataset import OID_Dataset as DS

### load data

In [2]:
__ds_root__ = '../google_oid_v6'
__train_f__ = 'oidv6-train-annotations-bbox.csv'
__test_f__ = '../google_oid_v6/ANNOT/test-annotations-bbox.csv'
__val_f__ = '../google_oid_v6/ANNOT/validation-annotations-bbox.csv'

ind = 0

classes = set(('Person', 'Car', 'Mobile phone'))
labels = pd.read_csv(f'{__ds_root__}/ANNOT/class-descriptions-boxable.csv')
label_ids = set(labels[labels.iloc[:,1].isin(classes)].iloc[:, 0])

In [38]:
# use custom collate, variable # of boxes in each image
def var_collate(batch):
    data = torch.tensor([item[0] for item in batch])
    target = [item[1] for item in batch]
#     target = torch.tensor(target)
    return data, target

oid_ds = DS(__ds_root__, __train_f__, label_ids)
train_load = DL(dataset=oid_ds, batch_size=4, shuffle=True, num_workers=4, collate_fn=var_collate)

In [31]:
class VGG(nn.Module):
    def __init__(self, c=64):
        super(VGG, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, c, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c, c, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(c, c*2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*2, c*2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(c*2, c*4, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*4, c*4, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*4, c*4, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(c*4, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        
    def forward(self, img):
        h = self.conv1(img)
        h = self.conv2(h)
        h = self.conv3(h)
        h = self.conv4(h)
        h = self.conv5(h)
        
        return h
    
class RPN(nn.Module):
    def __init__(self, c=512, ratio=[1.0, 2.0, 0.5]):
        super(RPN, self).__init__()
#         self.feat = nn.Conv2d(c, c/2, 3, stride=1, padding=1)
#         self.objness = nn.Conv2d()
    def forward(self, feat):
        pass
    
class YOLO_FC(nn.Module):
    def __init__(self, c=512):
        super(YOLO_FC, self).__init__()
        self.conn = nn.Sequential(
            nn.Linear(7 * 7 * 512, 4096),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(),
            nn.LeakyReLU(4096, 1470),
        )
    def forward(feat):
        return self.conn(feat)


In [39]:
conv_net = VGG().cuda(ind)
yolo_net = YOLO_FC().cuda(ind)

for img, y in train_load:
    img = img.cuda(ind)
    set_trace()
    o = conv_net(img)
    o = yolo_net(o)
#     set_trace()


ValueError: Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-38-514ee94ce62a>", line 3, in var_collate
    data = torch.tensor([item[0] for item in batch])
ValueError: only one element tensors can be converted to Python scalars


In [ ]:
%debug

[(0, 'Car'), (1, 'Person'), (2, 'Mobile phone')]
